# Punto 2
Un vendedor debe hacer un recorrido por todas y cada una de las capitales de los 32 estados de los Estados Unidos Mexicanos. 

Utilice colonias de hormigas y algoritmos genéticos para encontrar el orden óptimo. El costo de desplazamiento entre ciudades es la suma del valor de la hora del vendedor (es un parámetro que debe estudiarse), el costo de los peajes y el costo del combustible. Cada equipo debe definir en qué carro hace el recorrido el vendedor y de allí extraer el costo del combustible.

Adicionalmente represente con un gif animado o un video cómo se comporta la mejor solución usando un gráfico del recorrido en el mapa de México.

Primero, procederemos a cargar las librerías necesarias para abordar el problema

In [1]:
import pandas as pd
import folium
import numpy as np
from IPython.display import display
import random
from folium import plugins

Luego de esto, cargaremos algunos archivos necesarios, estos son:
1. Tiempos de desplazamiento entre estados: Estos datos serán necesarios para calcular el valor del pago al transportista, según también una constante que es el valor de la hora.
2. Coordenadas: Las coordenadas serán usadas para graficar el grafo que representa la ruta más óptima. Estas coordenadas corresponden a las capitales de cada Estado, se usan estas y no otras ciudades debido a que son precisamente las ciudades más importantes.
3. Distancias: Estas distancias se usarán para calcular el valor del consumo de combustible, según también una constante que es el costo de llenar un tanque de un vehículo que seleccionaremos. 
5. Peajes: Los peajes se incluyen también dentro del costo total del viaje de un estado a otro.

In [2]:
tiempos = pd.read_csv('./data/input/tabla_tiempos.csv',index_col=0)
distancias = pd.read_csv('./data/input/tabla_distancias.csv',index_col=0)
coordenadas = pd.read_csv('./data/input/coordenadas_cap_mex.csv',header=None)
coordenadas = coordenadas.rename(columns={0:'latitude', 1:'longitude'})
nombres_estados = list(tiempos.columns)
peajes = pd.read_csv('./data/input/peajes.csv',index_col=0)

In [3]:
def tiempo_dec(tiempo_str):
    if type(tiempo_str) == str:
        hh,mm = map(int, tiempo_str.split(':'))
        return round(hh+mm/60.0,4)
    else:
        return 0
tiempos = tiempos.fillna(0)
tiempos
tiempos_ = tiempos.copy().map(tiempo_dec)
times = tiempos_.to_numpy()


## Cálculo de variables de interés

### Valor de la hora * Tiempo de desplazamiento
Teniendo en cuenta que ya tenemos los tiempos de desplazamiento del vendedor, debemos crear la variable que multiplica esto por el valor de la hora. Para ello consultamos los salarios mínimos de 2024 diarios para cada tipo de profesión y oficio en esta página: https://www.gob.mx/cms/uploads/attachment/file/873886/Tabla_de_Salarios_M_nimos_2024.pdf, donde encontramos que el salario para un chofer de camioneta de carga en general (que es el trabajo de nuestro repartidor) es de \$284.76 MXN. El número de horas de trabajo usuales en México es de 8 horas, por lo que el salario mínimo por hora para el chofer es \$35.6 MXN aproximadamente. De esta manera, multiplicaremos este valor por el tiempo de desplazamiento entre estados:

In [4]:
val_hora_tiempo = times*35.6
print(val_hora_tiempo)

[[   0.      1050.79452 1041.89452 ...  439.66    1062.06548   72.98   ]
 [1030.02548    0.       921.44548 ... 1603.78    2225.59452  968.91452]
 [1069.78     913.73452    0.      ... 1418.06548 2040.47452  906.61452]
 ...
 [ 406.43452 1612.68    1431.12    ...    0.       712.59452  479.41452]
 [1017.56548 2223.81452 2042.25452 ...  704.28548    0.      1090.54548]
 [  69.42     984.93452  881.1     ...  508.48548 1130.3        0.     ]]


In [5]:
# exportar arreglo de numpy val_hora_tiempo a csv, redondeando a 2 decimales
np.savetxt('./data/output/val_hora_tiempo.csv', val_hora_tiempo, delimiter=',', fmt='%.2f')

### Costo del combustible

En este caso, elegimos una camioneta Chevrolet N400 MAX, elegido debido a que es un vehículo con la que se suelen repartir paquetes. Esta tiene un tanque con capacidad de 45 litros para gasolina (https://www.chevrolet.cl/content/dam/chevrolet/south-america/chile/espanol/index/fichas-tecnicas/03-pdf/ficha-tecnica-n400.pdf). El rendimiento del combustimble de este vehículo es de 16.1 km/l en carretera (https://www.latercera.com/mtonline/noticia/chevrolet-n400-max/988955-2/#:~:text=Asimismo%2C%20ahora%20est%C3%A1%20ubicado%20en,1%20km%2Fl%20en%20carretera.).

El precio de la gasolina (Magna en este caso) ronda los \$24 MXN por litro. Teniendo esta información en cuenta, procederemos a calcular esta variable, de esta manera:

$Costo \ combustible = \dfrac{distancia \ de \ un \ estado \ a \ otro}{eficiencia \ del \ combustible}*precio \ de \ la \ gasolina$

Donde la eficiencia del combustible y el precio de la gasolina son escalares, y se multiplican a nuestra matriz de distancias de un estado a otro.

In [6]:
distancias = distancias.fillna(0)
distancias_np = distancias.to_numpy()

cost_combustible = (distancias_np/16.1)*24
cost_combustible

array([[   0.        , 3206.45962733, 1996.02484472, ..., 1109.06832298,
        2657.88819876,  175.90062112],
       [3200.49689441,    0.        , 2018.38509317, ..., 4248.44720497,
        5797.26708075, 3029.06832298],
       [1996.02484472, 2018.38509317,    0.        , ..., 2859.13043478,
        4407.95031056, 1764.9689441 ],
       ...,
       [1103.10559006, 4245.46583851, 2856.14906832, ...,    0.        ,
        1665.0931677 , 1280.49689441],
       [2668.32298137, 5809.19254658, 4421.36645963, ..., 1663.60248447,
           0.        , 2844.22360248],
       [ 174.40993789, 3039.50310559, 1770.93167702, ..., 1283.47826087,
        2830.80745342,    0.        ]])

In [7]:
np.savetxt('./data/output/cost_combustible.csv', cost_combustible, delimiter=',', fmt='%.2f')

## Peajes
El costo de los peajes de un estado a otro fueron consultados en esta página, que está actualizada a este 2024, y fueron consultados teniendo en cuenta el tipo de vehículo que se usa en este caso (auto) https://tarifascapufe.com.mx/traza-tu-ruta/. Esta variable no es necesario hacerle alguna transformación, pues ya se encuentra en la unidad en la que están el resto (\$MXN).

In [8]:
peajes = peajes.fillna(0)
peajes_np = peajes.to_numpy()
peajes_np

array([[   0., 1588., 1629., ..., 1597., 2737.,   44.],
       [1588.,    0.,    0., ..., 5078., 6218., 1486.],
       [1629.,    0.,    0., ..., 4143., 5283., 1527.],
       ...,
       [1597., 5078., 4143., ...,    0., 1232.,  947.],
       [2737., 6218., 5283., ..., 1232.,    0., 2171.],
       [  44., 1486., 1527., ...,  947., 2171.,    0.]])

## Matriz final de costos
Teniendo ya las tres variables de interés para calcular los costos de ir de un estado a otro (Valor de la hora del transportista*tiempo de viaje de un estado a otro, costos de gasolina, y costos del peaje), procederemos a calcular nuestra matriz final de costos, la cual se calcula de la siguiente manera:

$Costo \ de \ ir \ de \ un \ estado \ a \ otro = Valor \ de \ la \ hora \ del \ transportista*tiempo \ de \ viaje \ de \ un \ estado \ a \ otro + costos \ de \ gasolina + costos \ de \ peajes$

Claramente todos los valores de nuestra matriz quedan en pesos mexicanos.

In [9]:
matriz_de_costos_final = val_hora_tiempo + cost_combustible + peajes_np
print(matriz_de_costos_final)
np.savetxt('./data/output/matriz_final.csv', matriz_de_costos_final, delimiter=',', fmt='%.2f')

[[    0.          5845.25414733  4666.91936472 ...  3145.72832298
   6456.95367876   292.88062112]
 [ 5818.52237441     0.          2939.83057317 ... 10930.22720497
  14240.86160075  5483.98284298]
 [ 4694.80484472  2932.11961317     0.         ...  8420.19591478
  11731.42483056  4198.5834641 ]
 ...
 [ 3106.54011006 10936.14583851  8430.26906832 ...     0.
   3609.6876877   2706.91141441]
 [ 6422.88846137 14251.00706658 11746.62097963 ...  3599.88796447
      0.          6105.76908248]
 [  287.82993789  5510.43762559  4179.03167702 ...  2738.96374087
   6132.10745342     0.        ]]


## Optimización del recorrido usando colonia de hormigas

In [10]:
import random
import numpy as np

random.seed(42)
np.random.seed(42)

# Parámetros del algoritmo
n_states = 32  # Número de estados
alpha = 1      # Influencia de las feromonas
beta = 2       # Influencia de la visibilidad (tiempos de viaje)
evaporation_rate = 0.5
pheromone_constant = 100
n_ants = n_states  # Usar una hormiga por estado

# Inicializa la matriz de feromonas
pheromones = np.ones((n_states, n_states))  # Cantidad inicial de feromonas

# Cálculo de visibilidad (inversa del tiempo de viaje, para que tiempos menores sean más atractivos)
visibility = 1 / matriz_de_costos_final
visibility[visibility == np.inf] = 0  # Evitar divisiones por cero

# Ejecución del algoritmo
best_route = None
best_cost = float('inf')

for iteration in range(100):  # Número de iteraciones para el algoritmo
    all_routes = []
    all_costs = []
    
    # Para cada hormiga
    for ant in range(n_ants):
        # Escoge un estado inicial aleatorio para la hormiga
        current_state = random.randint(0, n_states - 1)
        visited_states = [current_state]
        cost = 0
        
        # Construcción de la ruta completa
        for step in range(n_states - 1):
            # Calcular probabilidades para el próximo estado
            probabilities = []
            for next_state in range(n_states):
                if next_state not in visited_states:
                    prob = (pheromones[current_state][next_state] ** alpha) * (visibility[current_state][next_state] ** beta)
                    probabilities.append((next_state, prob))
            
            # Seleccionar el próximo estado basado en las probabilidades
            next_state = random.choices(
                [state for state, _ in probabilities],
                weights=[prob for _, prob in probabilities]
            )[0]
            
            # Actualizar el costo y el estado actual
            cost += matriz_de_costos_final[current_state][next_state]
            current_state = next_state
            visited_states.append(current_state)
        
        # Completa el ciclo regresando al punto inicial
        cost += matriz_de_costos_final[current_state][visited_states[0]]
        all_routes.append(visited_states)
        all_costs.append(cost)
        
        # Actualización de la mejor ruta
        if cost < best_cost:
            best_cost = cost
            best_route = visited_states

    # Actualización de feromonas
    pheromones *= (1 - evaporation_rate)  # Evaporación de feromonas
    for route, cost in zip(all_routes, all_costs):
        for i in range(n_states - 1):
            pheromones[route[i]][route[i + 1]] += pheromone_constant / cost
        # Retorno al punto inicial
        pheromones[route[-1]][route[0]] += pheromone_constant / cost

# Convertir la ruta a nombres de estados y mostrar el ciclo completo
best_route_names = [nombres_estados[i] for i in best_route]
# Agregar explícitamente el retorno al estado inicial
best_route_names.append(best_route_names[0])

print("\nMejor ruta encontrada (incluyendo retorno al inicio):")
print("→ ".join(best_route_names))

print("\nDesglose del costo por segmento:")
total_cost = 0
print("\n{:<35} {:<35} {:<15}".format("Origen", "Destino", "Costo (MXN)"))
print("-" * 85)

for i in range(len(best_route)):
    # Costo entre estados consecutivos
    current_state = best_route[i]
    next_state = best_route[0] if i == len(best_route) - 1 else best_route[i + 1]
    cost_segment = matriz_de_costos_final[current_state][next_state]
    total_cost += cost_segment
    print("{:<35} {:<35} ${:<15,.2f}".format(
        nombres_estados[current_state],
        nombres_estados[next_state],
        cost_segment
    ))

print("\nEstadísticas del viaje:")
print(f"Número total de estados visitados: {len(best_route_names)} (incluyendo el retorno al inicio)")
print(f"Costo total del viaje: ${total_cost:,.2f} MXN")
print(f"Costo promedio por segmento: ${total_cost/len(best_route):,.2f} MXN")


C:\Users\aoroz\AppData\Local\Temp\ipykernel_14208\2398117130.py:19: RuntimeWarning: divide by zero encountered in divide
  visibility = 1 / matriz_de_costos_final



Mejor ruta encontrada (incluyendo retorno al inicio):
Morelia→ Guadalajara→ Colima→ Tepic→ Culiacán Rosales→ La Paz→ Hermosillo→ Mexicali→ Chihuahua→ Victoria de Durango→ Zacatecas→ Aguascalientes→ San Luis Potosí→ Ciudad Victoria→ Monterrey→ Saltillo→ Guanajuato→ Santiago de Querétaro→ Toluca de Lerdo→ CDMX→ Cuernavaca→ Chilpancingo de los Bravo→ Heroica Puebla de Zaragoza→ Oaxaca de Juárez→ Tuxtla Gutiérrez→ Villahermosa→ San Francisco de Campeche→ Mérida→ Chetumal→ Xalapa-Enríquez→ Tlaxcala de Xicohténcatl→ Pachuca de Soto→ Morelia

Desglose del costo por segmento:

Origen                              Destino                             Costo (MXN)    
-------------------------------------------------------------------------------------
Morelia                             Guadalajara                         $1,205.48       
Guadalajara                         Colima                              $887.90         
Colima                              Tepic                              

In [11]:
coordenadas_estados = {estado: (lat, lon) for estado, lat, lon in zip(nombres_estados, coordenadas['latitude'], coordenadas['longitude'])}

In [13]:
import folium
import openrouteservice
from dotenv import load_dotenv
import os
load_dotenv()

coordenadas_estados =  {key: coordenadas_estados[key] for key in best_route_names if key in coordenadas_estados}

client = openrouteservice.Client(key=os.getenv("ORS_TOKEN"))

primera_ciudad = list(coordenadas_estados.values())[0]
m = folium.Map(location=primera_ciudad, zoom_start=6)

ciudades = list(coordenadas_estados.keys())
for i in range(len(ciudades) - 1):
    origen = coordenadas_estados[ciudades[i]]
    destino = coordenadas_estados[ciudades[i + 1]]

    try:
        # Calcular la ruta entre origen y destino
        ruta = client.directions(
            coordinates=[origen[::-1], destino[::-1]],  # Revertimos a (longitud, latitud)
            profile='driving-car',
            format='geojson',
            radiuses=[1000, 1000]
        )

        # Dibujar la ruta en el mapa
        folium.GeoJson(ruta, name=f'Ruta {ciudades[i]} a {ciudades[i + 1]}').add_to(m)

        # Agregar marcadores de las ciudades
        folium.Marker(location=origen, popup=ciudades[i], icon=folium.Icon(color='red')).add_to(m)
        folium.Marker(location=destino, popup=ciudades[i + 1], icon=folium.Icon(color='red')).add_to(m)

    except openrouteservice.exceptions.ApiError as e:
        print(f"Error al calcular la ruta entre {ciudades[i]} y {ciudades[i + 1]}: {e}")
        
ruta = client.directions(
            coordinates=[primera_ciudad[::-1], destino[::-1]],
            profile='driving-car',
            format='geojson',
            radiuses=[1000, 1000]
        )

folium.GeoJson(ruta, name=f'Ruta {ciudades[i]} a {ciudades[i + 1]}').add_to(m)

folium.Marker(location=primera_ciudad, popup=list(coordenadas_estados.keys())[0], icon=folium.Icon(color='green')).add_to(m)
folium.Marker(location=list(coordenadas_estados.values())[-1], popup=list(coordenadas_estados.keys())[-1], icon=folium.Icon(color='green')).add_to(m)

# Guardar el mapa como un archivo HTML
m.save("./data/output/rutas_colonia_hormigas.html")
print("El mapa se ha guardado como './data/output/rutas_colonia_hormigas.html'")
display(m)


ValueError: No API key was specified. Please visit https://openrouteservice.org/sign-up to create one.

In [12]:
import folium
from folium.plugins import AntPath

# Coordenadas aproximadas de cada estado en formato (lat, lon) - ejemplo de coordenadas en México
#crear diccionario apartir de la lista de coordenadas
coordenadas_estados = {estado: (lat, lon) for estado, lat, lon in zip(nombres_estados, coordenadas['latitude'], coordenadas['longitude'])}


# Ruta del recorrido en orden
ruta = best_route_names  # Reemplaza con tu ruta optimizada

# Inicializar el mapa centrado en México
m = folium.Map(location=[23.6345, -102.5528], zoom_start=5)

inicio_coord = coordenadas_estados[ruta[0]]
folium.Marker(
    location=inicio_coord,
    popup="Inicio",
    icon=folium.Icon(color="green", icon="play")
).add_to(m)

# Añadir un marcador especial para el fin (último estado en la ruta)
fin_coord = coordenadas_estados[ruta[-1]]
folium.Marker(
    location=fin_coord,
    popup="Fin",
    icon=folium.Icon(color="green", icon="play")
).add_to(m)


for estado in ruta[1:-1]:  # Excluir inicio y fin para evitar superposición
    coord = coordenadas_estados[estado]
    folium.Marker(location=coord, popup=f"Estado {estado}").add_to(m)

# Construir la ruta animada con AntPath
ruta_coords = [coordenadas_estados[estado] for estado in ruta]
AntPath(ruta_coords, color="blue", weight=4, opacity=0.8).add_to(m)
m

## Optimización usando algoritmos genéticos

In [13]:
import numpy as np
import random

random.seed(42)
np.random.seed(42)

# Parámetros del problema
n_states = 32  # Número de estados
n_population = 100  # Tamaño de la población
n_generations = 200  # Número de generaciones
mutation_rate = 0.01  # Tasa de mutación

# Genera una ruta aleatoria
def generate_route():
    route = list(range(n_states))
    random.shuffle(route)
    return route

# Calcula el costo total de una ruta
def calculate_cost(route):
    total_cost = 0
    segments = []
    
    # Calcula el costo entre cada par de estados consecutivos
    for i in range(n_states - 1):
        cost = matriz_de_costos_final[route[i], route[i + 1]]
        total_cost += cost
        segments.append((route[i], route[i + 1], cost))
    
    # Añade el costo del retorno al estado inicial
    cost_return = matriz_de_costos_final[route[-1], route[0]]
    total_cost += cost_return
    segments.append((route[-1], route[0], cost_return))
    
    return total_cost, segments

# Genera la población inicial
population = [generate_route() for _ in range(n_population)]

# Seguimiento de la mejor ruta en cada generación
best_costs_per_generation = []

# Algoritmo evolutivo
for generation in range(n_generations):
    # Evaluación de la aptitud
    fitness_scores = [(route, *calculate_cost(route)) for route in population]
    fitness_scores.sort(key=lambda x: x[1])  # Ordena por costo
    
    # Guarda el mejor costo de esta generación
    best_costs_per_generation.append(fitness_scores[0][1])

    # Selección (elitismo: conserva las mejores rutas)
    elite_size = int(0.1 * n_population)  # Mantener el 10% de la mejor población
    new_population = [route for route, _, _ in fitness_scores[:elite_size]]
    
    # Generación de nuevos individuos (crossover y mutación)
    while len(new_population) < n_population:
        # Selección de padres
        parent1, parent2 = random.choices(population, k=2)
        
        # Crossover (Orden basado en cruce)
        crossover_point = random.randint(1, n_states - 2)
        child = parent1[:crossover_point] + [state for state in parent2 if state not in parent1[:crossover_point]]
        
        # Mutación
        if random.random() < mutation_rate:
            i, j = random.sample(range(n_states), 2)
            child[i], child[j] = child[j], child[i]  # Intercambia dos estados aleatorios
        
        new_population.append(child)
    
    population = new_population  # Actualiza la población para la siguiente generación

# Mejor ruta encontrada
best_route, best_cost, best_segments = min(fitness_scores, key=lambda x: x[1])
best_route_names = [nombres_estados[i] for i in best_route]
# Agregar explícitamente el retorno al estado inicial
best_route_names.append(best_route_names[0])

# Mostrar resultados
print("\nMejor ruta encontrada (incluyendo retorno al inicio):")
print("→ ".join(best_route_names))

print("\nDesglose del costo por segmento:")
print("\n{:<35} {:<35} {:<15}".format("Origen", "Destino", "Costo (MXN)"))
print("-" * 85)

for origin, destination, cost in best_segments:
    print("{:<35} {:<35} ${:<15,.2f}".format(
        nombres_estados[origin],
        nombres_estados[destination],
        cost
    ))

print("\nEstadísticas del viaje:")
print(f"Número total de estados visitados: {len(best_route_names)} (incluyendo el retorno al inicio)")
print(f"Costo total del viaje: ${best_cost:,.2f} MXN")
print(f"Costo promedio por segmento: ${best_cost/len(best_route):,.2f} MXN")

# Estadísticas de la evolución
print("\nEstadísticas de la evolución:")
print(f"Costo inicial (mejor de la primera generación): ${best_costs_per_generation[0]:,.2f} MXN")
print(f"Costo final (mejor de la última generación): ${best_costs_per_generation[-1]:,.2f} MXN")
print(f"Mejora total: ${(best_costs_per_generation[0] - best_costs_per_generation[-1]):,.2f} MXN")
print(f"Porcentaje de mejora: {((best_costs_per_generation[0] - best_costs_per_generation[-1]) / best_costs_per_generation[0] * 100):,.2f}%")


Mejor ruta encontrada (incluyendo retorno al inicio):
Zacatecas→ Cuernavaca→ Saltillo→ Monterrey→ Ciudad Victoria→ San Luis Potosí→ Aguascalientes→ Guadalajara→ Culiacán Rosales→ Hermosillo→ Tepic→ Colima→ Guanajuato→ Pachuca de Soto→ Morelia→ Santiago de Querétaro→ Tlaxcala de Xicohténcatl→ CDMX→ Chilpancingo de los Bravo→ Chetumal→ Mérida→ San Francisco de Campeche→ Tuxtla Gutiérrez→ Villahermosa→ Xalapa-Enríquez→ Oaxaca de Juárez→ Heroica Puebla de Zaragoza→ Toluca de Lerdo→ Victoria de Durango→ Chihuahua→ Mexicali→ La Paz→ Zacatecas

Desglose del costo por segmento:

Origen                              Destino                             Costo (MXN)    
-------------------------------------------------------------------------------------
Zacatecas                           Cuernavaca                          $1,962.49       
Cuernavaca                          Saltillo                            $2,731.64       
Saltillo                            Monterrey                        

In [14]:
coordenadas_estados = {estado: (lat, lon) for estado, lat, lon in zip(nombres_estados, coordenadas['latitude'], coordenadas['longitude'])}

In [ ]:
import folium
import openrouteservice
from dotenv import load_dotenv
import os
load_dotenv()
coordenadas_estados = {key: coordenadas_estados[key] for key in best_route_names if key in coordenadas_estados}

# Inicializar cliente de OpenRouteService
client = openrouteservice.Client(key=os.getenv("ORS_TOKEN"))

# Crear el mapa centrado en la primera ciudad
primera_ciudad = list(coordenadas_estados.values())[0]
m = folium.Map(location=primera_ciudad, zoom_start=6)

# Obtener las rutas y agregarlas al mapa
ciudades = list(coordenadas_estados.keys())
for i in range(len(ciudades) - 1):
    origen = coordenadas_estados[ciudades[i]]
    destino = coordenadas_estados[ciudades[i + 1]]

    try:
        # Calcular la ruta entre origen y destino
        ruta = client.directions(
            coordinates=[origen[::-1], destino[::-1]],
            profile='driving-car',
            format='geojson',
            radiuses=[1000, 1000]
        )

        # Dibujar la ruta en el mapa
        folium.GeoJson(ruta, name=f'Ruta {ciudades[i]} a {ciudades[i + 1]}').add_to(m)

        # Agregar marcadores de las ciudades
        folium.Marker(location=origen, popup=ciudades[i], icon=folium.Icon(color='blue')).add_to(m)
        folium.Marker(location=destino, popup=ciudades[i + 1], icon=folium.Icon(color='blue')).add_to(m)

    except openrouteservice.exceptions.ApiError as e:
        print(f"Error al calcular la ruta entre {ciudades[i]} y {ciudades[i + 1]}: {e}")
ruta = client.directions(
            coordinates=[primera_ciudad[::-1], destino[::-1]],
            profile='driving-car',
            format='geojson',
            radiuses=[1000, 1000]
        )

        # Dibujar la ruta en el mapa
folium.GeoJson(ruta, name=f'Ruta {ciudades[i]} a {ciudades[i + 1]}').add_to(m)
folium.Marker(location=primera_ciudad, popup=list(coordenadas_estados.keys())[0], icon=folium.Icon(color='green')).add_to(m)
folium.Marker(location=list(coordenadas_estados.values())[-1], popup=list(coordenadas_estados.keys())[-1], icon=folium.Icon(color='green')).add_to(m)

# Guardar el mapa como un archivo HTML
m.save("./data/output/rutas_algoritmo_genetico.html")
print("El mapa se ha guardado como './data/output/rutas_algoritmo_genetico.html'")
display(m)


c:\Users\JUAN\Documents\Universidad 2024-2\Redes neuronales\Repos\Primer-trabajo-rnyabi\.venv\Lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
c:\Users\JUAN\Documents\Universidad 2024-2\Redes neuronales\Repos\Primer-trabajo-rnyabi\.venv\Lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
c:\Users\JUAN\Documents\Universidad 2024-2\Redes neuronales\Repos\Primer-trabajo-rnyabi\.venv\Lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
c:\Users\JUAN\Documents\Universidad 2024-2\Redes neuronales\Repos\Primer-trabajo-rnyabi\.venv\Lib\si

Error al calcular la ruta entre CDMX y Chilpancingo de los Bravo: 404 ({'error': {'code': 2010, 'message': 'Could not find routable point within a radius of 1000.0 meters of specified coordinate 1: -99.7428629 17.3919748.'}, 'info': {'engine': {'build_date': '2024-10-09T09:23:42Z', 'version': '8.2.0'}, 'timestamp': 1731984683946}})
Error al calcular la ruta entre Chilpancingo de los Bravo y Chetumal: 404 ({'error': {'code': 2010, 'message': 'Could not find routable point within a radius of 1000.0 meters of specified coordinate 0: -99.7428629 17.3919748.'}, 'info': {'engine': {'build_date': '2024-10-09T09:23:42Z', 'version': '8.2.0'}, 'timestamp': 1731984684161}})
El mapa se ha guardado como './data/output/rutas_algoritmo_genetico.html'


In [15]:
import folium
from folium.plugins import AntPath

# Coordenadas aproximadas de cada estado en formato (lat, lon) - ejemplo de coordenadas en México
#crear diccionario apartir de la lista de coordenadas
coordenadas_estados = {estado: (lat, lon) for estado, lat, lon in zip(nombres_estados, coordenadas['latitude'], coordenadas['longitude'])}
# Ruta del recorrido en orden
ruta = best_route_names  # Reemplaza con tu ruta optimizada

# Inicializar el mapa centrado en México
m = folium.Map(location=[23.6345, -102.5528], zoom_start=5)

inicio_coord = coordenadas_estados[ruta[0]]
folium.Marker(
    location=inicio_coord,
    popup="Inicio",
    icon=folium.Icon(color="green", icon="play")
).add_to(m)

# Añadir un marcador especial para el fin (último estado en la ruta)
fin_coord = coordenadas_estados[ruta[-1]]
folium.Marker(
    location=fin_coord,
    popup="Fin",
    icon=folium.Icon(color="green", icon="play")
).add_to(m)


for estado in ruta[1:-1]:  # Excluir inicio y fin para evitar superposición
    coord = coordenadas_estados[estado]
    folium.Marker(location=coord, popup=f"Estado {estado}").add_to(m)

# Construir la ruta animada con AntPath
ruta_coords = [coordenadas_estados[estado] for estado in ruta]
AntPath(ruta_coords, color="blue", weight=4, opacity=0.8).add_to(m)
m

Haciendo una comparación entre ambos algoritmos, vemos que el de colonia de hormigas optimiza de una mejor manera los costos de transporte en la mejor ruta que los algoritmos genéticos, esto lo podemos ver en el costo final de cada ruta en cada algoritmo, para colonia de hormigas obtuvimos un costo de \$36590.41 MXN, mientras que para los algoritmos genéticos se obtiene un costo total de \$57689.98 MXN

# Usando el mismo punto de partida

## Algoritmos genéticos

In [20]:
import numpy as np
import random

random.seed(42)
np.random.seed(42)

# Parámetros del problema
n_states = 32  # Número de estados
n_population = 100  # Tamaño de la población
n_generations = 200  # Número de generaciones
mutation_rate = 0.01  # Tasa de mutación

# Genera una ruta aleatoria comenzando desde Ciudad de México (índice 0)
def generate_route():
    route = list(range(1, n_states))  # Empieza con los demás estados (excepto Ciudad de México)
    random.shuffle(route)
    route = [0] + route  # Inserta Ciudad de México (índice 0) al inicio
    return route

# Calcula el costo total de una ruta
def calculate_cost(route):
    total_cost = 0
    segments = []
    
    # Calcula el costo entre cada par de estados consecutivos
    for i in range(n_states - 1):
        cost = matriz_de_costos_final[route[i], route[i + 1]]
        total_cost += cost
        segments.append((route[i], route[i + 1], cost))
    
    # Añade el costo del retorno al estado inicial
    cost_return = matriz_de_costos_final[route[-1], route[0]]
    total_cost += cost_return
    segments.append((route[-1], route[0], cost_return))
    
    return total_cost, segments

# Genera la población inicial
population = [generate_route() for _ in range(n_population)]

# Seguimiento de la mejor ruta en cada generación
best_costs_per_generation = []

# Algoritmo evolutivo
for generation in range(n_generations):
    # Evaluación de la aptitud
    fitness_scores = [(route, *calculate_cost(route)) for route in population]
    fitness_scores.sort(key=lambda x: x[1])  # Ordena por costo
    
    # Guarda el mejor costo de esta generación
    best_costs_per_generation.append(fitness_scores[0][1])

    # Selección (elitismo: conserva las mejores rutas)
    elite_size = int(0.1 * n_population)  # Mantener el 10% de la mejor población
    new_population = [route for route, _, _ in fitness_scores[:elite_size]]
    
    # Generación de nuevos individuos (crossover y mutación)
    while len(new_population) < n_population:
        # Selección de padres
        parent1, parent2 = random.choices(population, k=2)
        
        # Crossover (Orden basado en cruce)
        crossover_point = random.randint(1, n_states - 2)
        child = parent1[:crossover_point] + [state for state in parent2 if state not in parent1[:crossover_point]]
        
        # Mutación
        if random.random() < mutation_rate:
            i, j = random.sample(range(n_states), 2)
            child[i], child[j] = child[j], child[i]  # Intercambia dos estados aleatorios
        
        new_population.append(child)
    
    population = new_population  # Actualiza la población para la siguiente generación

# Mejor ruta encontrada
best_route, best_cost, best_segments = min(fitness_scores, key=lambda x: x[1])
best_route_names = [nombres_estados[i] for i in best_route]
# Agregar explícitamente el retorno al estado inicial
best_route_names.append(best_route_names[0])

# Mostrar resultados
print("\nMejor ruta encontrada (incluyendo retorno al inicio):")
print("→ ".join(best_route_names))

print("\nDesglose del costo por segmento: ")
print("\n{:<35} {:<35} {:<15}".format("Origen", "Destino", "Costo (MXN)"))
print("-" * 85)

for origin, destination, cost in best_segments:
    print("{:<35} {:<35} ${:<15,.2f}".format(
        nombres_estados[origin],
        nombres_estados[destination],
        cost
    ))

print("\nEstadísticas del viaje:")
print(f"Número total de estados visitados: {len(best_route_names)} (incluyendo el retorno al inicio)")
print(f"Costo total del viaje: ${best_cost:,.2f} MXN")
print(f"Costo promedio por segmento: ${best_cost/len(best_route):,.2f} MXN")

# Estadísticas de la evolución
print("\nEstadísticas de la evolución:")
print(f"Costo inicial (mejor de la primera generación): ${best_costs_per_generation[0]:,.2f} MXN")
print(f"Costo final (mejor de la última generación): ${best_costs_per_generation[-1]:,.2f} MXN")
print(f"Mejora total: ${(best_costs_per_generation[0] - best_costs_per_generation[-1]):,.2f} MXN")
print(f"Porcentaje de mejora: {((best_costs_per_generation[0] - best_costs_per_generation[-1]) / best_costs_per_generation[0] * 100):,.2f}%")



Mejor ruta encontrada (incluyendo retorno al inicio):
Aguascalientes→ Santiago de Querétaro→ CDMX→ Toluca de Lerdo→ Guadalajara→ Colima→ Morelia→ Chilpancingo de los Bravo→ Heroica Puebla de Zaragoza→ Oaxaca de Juárez→ Tuxtla Gutiérrez→ Villahermosa→ Tlaxcala de Xicohténcatl→ Guanajuato→ Victoria de Durango→ Culiacán Rosales→ La Paz→ Chihuahua→ Saltillo→ San Luis Potosí→ San Francisco de Campeche→ Mérida→ Chetumal→ Xalapa-Enríquez→ Pachuca de Soto→ Cuernavaca→ Tepic→ Hermosillo→ Mexicali→ Monterrey→ Ciudad Victoria→ Zacatecas→ Aguascalientes

Desglose del costo por segmento: 

Origen                              Destino                             Costo (MXN)    
-------------------------------------------------------------------------------------
Aguascalientes                      Santiago de Querétaro               $1,244.42       
Santiago de Querétaro               CDMX                                $798.88         
CDMX                                Toluca de Lerdo            

In [21]:
import folium
from folium.plugins import AntPath

# Coordenadas aproximadas de cada estado en formato (lat, lon) - ejemplo de coordenadas en México
#crear diccionario apartir de la lista de coordenadas
coordenadas_estados = {estado: (lat, lon) for estado, lat, lon in zip(nombres_estados, coordenadas['latitude'], coordenadas['longitude'])}


# Ruta del recorrido en orden
ruta = best_route_names  # Reemplaza con tu ruta optimizada

# Inicializar el mapa centrado en México
m = folium.Map(location=[23.6345, -102.5528], zoom_start=5)

inicio_coord = coordenadas_estados[ruta[0]]
folium.Marker(
    location=inicio_coord,
    popup="Inicio",
    icon=folium.Icon(color="green", icon="play")
).add_to(m)

# Añadir un marcador especial para el fin (último estado en la ruta)
fin_coord = coordenadas_estados[ruta[-1]]
folium.Marker(
    location=fin_coord,
    popup="Fin",
    icon=folium.Icon(color="green", icon="play")
).add_to(m)


for estado in ruta[1:-1]:  # Excluir inicio y fin para evitar superposición
    coord = coordenadas_estados[estado]
    folium.Marker(location=coord, popup=f"Estado {estado}").add_to(m)

# Construir la ruta animada con AntPath
ruta_coords = [coordenadas_estados[estado] for estado in ruta]
AntPath(ruta_coords, color="blue", weight=4, opacity=0.8).add_to(m)
m.save("./data/output/rutas_algoritmo_genetico_animado_partida_fija.html")
display(m)

## Colonia de hormigas

In [22]:
import random
import numpy as np

random.seed(42)
np.random.seed(42)

# Parámetros del algoritmo
n_states = 32  # Número de estados
alpha = 1      # Influencia de las feromonas
beta = 2       # Influencia de la visibilidad (tiempos de viaje)
evaporation_rate = 0.5
pheromone_constant = 100
n_ants = n_states  # Usar una hormiga por estado

# Inicializa la matriz de feromonas
pheromones = np.ones((n_states, n_states))  # Cantidad inicial de feromonas

# Cálculo de visibilidad (inversa del tiempo de viaje, para que tiempos menores sean más atractivos)
visibility = 1 / matriz_de_costos_final
visibility[visibility == np.inf] = 0  # Evitar divisiones por cero

# Ejecución del algoritmo
best_route = None
best_cost = float('inf')

for iteration in range(100):  # Número de iteraciones para el algoritmo
    all_routes = []
    all_costs = []
    
    # Para cada hormiga
    for ant in range(n_ants):
        # Fijar el estado inicial a Ciudad de México (índice 0)
        current_state = 0  # Ciudad de México
        visited_states = [current_state]
        cost = 0
        
        # Construcción de la ruta completa
        for step in range(n_states - 1):
            # Calcular probabilidades para el próximo estado
            probabilities = []
            for next_state in range(n_states):
                if next_state not in visited_states:
                    prob = (pheromones[current_state][next_state] ** alpha) * (visibility[current_state][next_state] ** beta)
                    probabilities.append((next_state, prob))
            
            # Seleccionar el próximo estado basado en las probabilidades
            next_state = random.choices(
                [state for state, _ in probabilities],
                weights=[prob for _, prob in probabilities]
            )[0]
            
            # Actualizar el costo y el estado actual
            cost += matriz_de_costos_final[current_state][next_state]
            current_state = next_state
            visited_states.append(current_state)
        
        # Completa el ciclo regresando al punto inicial
        cost += matriz_de_costos_final[current_state][visited_states[0]]
        all_routes.append(visited_states)
        all_costs.append(cost)
        
        # Actualización de la mejor ruta
        if cost < best_cost:
            best_cost = cost
            best_route = visited_states

    # Actualización de feromonas
    pheromones *= (1 - evaporation_rate)  # Evaporación de feromonas
    for route, cost in zip(all_routes, all_costs):
        for i in range(n_states - 1):
            pheromones[route[i]][route[i + 1]] += pheromone_constant / cost
        # Retorno al punto inicial
        pheromones[route[-1]][route[0]] += pheromone_constant / cost

# Convertir la ruta a nombres de estados y mostrar el ciclo completo
best_route_names = [nombres_estados[i] for i in best_route]
# Agregar explícitamente el retorno al estado inicial
best_route_names.append(best_route_names[0])

print("\nMejor ruta encontrada (incluyendo retorno al inicio):")
print("→ ".join(best_route_names))

print("\nDesglose del costo por segmento:")
total_cost = 0
print("\n{:<35} {:<35} {:<15}".format("Origen", "Destino", "Costo (MXN)"))
print("-" * 85)

for i in range(len(best_route)):
    # Costo entre estados consecutivos
    current_state = best_route[i]
    next_state = best_route[0] if i == len(best_route) - 1 else best_route[i + 1]
    cost_segment = matriz_de_costos_final[current_state][next_state]
    total_cost += cost_segment
    print("{:<35} {:<35} ${:<15,.2f}".format(
        nombres_estados[current_state],
        nombres_estados[next_state],
        cost_segment
    ))

print("\nEstadísticas del viaje:")
print(f"Número total de estados visitados: {len(best_route_names)} (incluyendo el retorno al inicio)")
print(f"Costo total del viaje: ${total_cost:,.2f} MXN")
print(f"Costo promedio por segmento: ${total_cost/len(best_route):,.2f} MXN")



C:\Users\aoroz\AppData\Local\Temp\ipykernel_14208\3130419368.py:19: RuntimeWarning: divide by zero encountered in divide
  visibility = 1 / matriz_de_costos_final



Mejor ruta encontrada (incluyendo retorno al inicio):
Aguascalientes→ Zacatecas→ San Luis Potosí→ Ciudad Victoria→ Monterrey→ Saltillo→ Guanajuato→ Santiago de Querétaro→ Toluca de Lerdo→ CDMX→ Cuernavaca→ Chilpancingo de los Bravo→ Pachuca de Soto→ Tlaxcala de Xicohténcatl→ Heroica Puebla de Zaragoza→ Oaxaca de Juárez→ Tuxtla Gutiérrez→ Villahermosa→ San Francisco de Campeche→ Mérida→ Chetumal→ Xalapa-Enríquez→ Morelia→ Guadalajara→ Colima→ Tepic→ Culiacán Rosales→ La Paz→ Hermosillo→ Mexicali→ Chihuahua→ Victoria de Durango→ Aguascalientes

Desglose del costo por segmento:

Origen                              Destino                             Costo (MXN)    
-------------------------------------------------------------------------------------
Aguascalientes                      Zacatecas                           $292.88         
Zacatecas                           San Luis Potosí                     $613.62         
San Luis Potosí                     Ciudad Victoria             

In [23]:
import folium
from folium.plugins import AntPath

# Coordenadas aproximadas de cada estado en formato (lat, lon) - ejemplo de coordenadas en México
#crear diccionario apartir de la lista de coordenadas
coordenadas_estados = {estado: (lat, lon) for estado, lat, lon in zip(nombres_estados, coordenadas['latitude'], coordenadas['longitude'])}


# Ruta del recorrido en orden
ruta = best_route_names  # Reemplaza con tu ruta optimizada

# Inicializar el mapa centrado en México
m = folium.Map(location=[23.6345, -102.5528], zoom_start=5)

inicio_coord = coordenadas_estados[ruta[0]]
folium.Marker(
    location=inicio_coord,
    popup="Inicio",
    icon=folium.Icon(color="green", icon="play")
).add_to(m)

# Añadir un marcador especial para el fin (último estado en la ruta)
fin_coord = coordenadas_estados[ruta[-1]]
folium.Marker(
    location=fin_coord,
    popup="Fin",
    icon=folium.Icon(color="green", icon="play")
).add_to(m)


for estado in ruta[1:-1]:  # Excluir inicio y fin para evitar superposición
    coord = coordenadas_estados[estado]
    folium.Marker(location=coord, popup=f"Estado {estado}").add_to(m)

# Construir la ruta animada con AntPath
ruta_coords = [coordenadas_estados[estado] for estado in ruta]
AntPath(ruta_coords, color="blue", weight=4, opacity=0.8).add_to(m)
m.save('./data/output/rutas_colonia_hormigas_animado_partida_fija.html')
display(m)